# Pipeline steps

### 1. Training set preparation
--------------------------------------------------------------------------
__1.1 PDB search__

Extract all the rappresentative proteins with the Kunitz Domain form `PDB`

- Use advanced search:
    - Data Collection Resolution <= 3.5
    - Identifier - Pfam Protein Family: PF00014
    - Polymer Entity Sequence Length <= 80 and >= 45

- Total of 160 protein


In [ ]:
# Change the .csv file in a .fasta file
cat  rcsb_pdb_custom_report_20250505053057.csv | tr -d '"' \
| awk -F ',' '{if (length($2)>0) {name=$2}; print name ,$3,$4,$5}' \
| grep PF00014 \
| awk '{print ">"$1"_"$3; print $2}' > pdb_kunitz_customreported.fasta

__2.2 Cluster identical sequences__
- `cd-hit` for clusterize and filter the redundancy

In [ ]:
# Keeping only one representative per cluster with ≥90% identity (-c 0.9)
cd-hit -i pdb_kunitz_customreported.fasta -o pdb_kunitz_customreported_nr.fasta -c 0.9

# I obtained 25 clusters and from this I removed the protein 20DY_E because too long
awk '/^>2ODY_E/ {getline; next} {print}' pdb_kunitz_customreported_nr.fasta > pdb_kunitz_customreported_filtered.fasta

# From the clstr file obtained from cd-hit generate a file with all the IDs
grep -v "^>" pdb_kunitz_customreported_filtered.clstr | sed -n 's/.*>\(.*\)\.\.\..*/\1/p' > pdb_kunitz_rp.ids.txt

# Extract all the fasta file of the corresponding clustered IDs using the customreported file as reference
for i in $(cat pdb_kunitz_rp.ids.txt); do
  grep -A 1 "^>$i" pdb_kunitz_customreported_filtered.fasta | head -n 2 >> pdb_kunitz_rp.fasta
done

# Extract sequence IDs, remove '>', replace '_' with ':', and save to file
grep ">" pdb_kunitz_rp.fasta | tr -d ">" | tr "_" ":" > tmp_pdb_efold_ids.txt

FILE :
- `pdb_kunitz_customreported_filtered.fasta`
- `tmp_pdb_efold_ids.txt`

### 2. Perform the MSA with PDBefold
--------------------------------------------------------------------------
A multiple sequence alignment (MSA) is performed manually using `PDBefold`. The MSA is a structure alignment and is used for the HMM training.
In the PDBefold output I remove a protein because of a RDSM grater than 1 Angstrom.

Download the file and format it to make it compatible with the HMM model



In [ ]:
# It converts all sequence characters to uppercase and ensures that the sequences are printed without any extra spaces or newlines, except for the header lines (starting with '>').
awk '{
  if (substr($1, 1, 1) == ">") {
    print "\n" toupper($1)
  } else {
    printf "%s", toupper($1)
  }
}' pdb_kunitz_rp.ali > pdb_kunitz_rp_formatted.ali


FILE : `pdb_kunitz_rp_formatted.ali`

### 3. HMM model construction
--------------------------------------------------------------------------

Using `hmmbuild` function of `HMMER (v3.3.2)` to train the HMM model for detecting the Kunitz Domain

In [ ]:
 # INPUT: pdb_kunitz_rp_formatted-ali
 # OUTPUT: structural_model.hmm
 hmmbuild structural_model.hmm pdb_kunitz_rp_formatted.ali

The constructed HMM contains 58 match states (on 84 positions) derived from conserved positions in the multiple sequence alignment.

FILE:
- `structural_model.hmm` (this is the obtained model)
- Sequence Logo images:
  - SL_HMM.png (obtained from `structural_model.hmm`)
  - SL_Ali.png (obtained form `pdb_kunitz_rp_formatted.ali`)

**In sequence logo there are 4 important conserved Cysteine fundamental for the Kunitz Domain structure**

### 4. Test sets preparation
--------------------------------------------------------------------------
__4.1 Positive set (with Kunitz Domain)__

1. First I manually obtain from Uniprot a file with all the proteins with Kunitz Domain, using the advance search:
  - domain: Kunitz
  - Pfam: PF00014

    I obtain 397 protein

    FILE: `all_kunitz.fasta` --> 397 proteins

2. Positive test sets was filtered for redundancy using `BLAST`, removing sequences with >=95% identity and >=50% coverage to each other or to training sequences, to ensure evaluation on non-overlapping data.



In [ ]:
# BLAST database creation
makeblastdb -in all_kunitz.fasta -dbtype prot -out all_kunitz.fasta

# Apply BLAST
blastp -query pdb_kunitz_rp.fasta -db all_kunitz.fasta -out pdb_kunitz_nr_23.blast -outfm 7

# Extract the Uniprot IDs with >=95% identity and >=50% coverage
grep -v "^#" pdb_kunitz_nr_23.blast \
  | awk '{ if ($3 >= 95 && $4 >= 50) print $2 }' \
  | sort -u \
  | cut -d "|" -f2 \
  > to_remove.ids

# Obtain all the IDs from the all_kunitz.fasta and compare them to the IDs to remove to erase them form the positive test set
grep ">" all_kunitz.fasta | cut -d "|" -f 2 > all_kunitz.id
comm -23 <(sort all_kunitz.id) <(sort to_remove.ids) > to_keep.ids

FILE:
- `pdb_kunitz_nr_23.blast`
- `to_remove.ids` --> 29 protein to remove
- `to_keep.ids` --> 368 protein

  3. Pull out from a fasta file only the sequences of the IDs list using a python script (`get_seq.py`)

In [ ]:
# IDs list --> to_keep.ids
# Fasta file --> all_kunitz.fasta
# Output file --> ok_kunitz.fasta
python3 get_seq.py to_keep.ids all_kunitz.fasta ok_kunitz.fasta

FILE: `ok_kunitz.fasta ` (this is the positive set)


__4.2 Negative test set (without Kunitz Domain)__

1. I select all the non Kunitz Domanin's protein from all the UniProtKB Reviewed (Swiss-Prot) available online form this link: https://www.uniprot.org/help/downloads

In [ ]:
grep ">" uniprot_sprot.fasta | cut -d "|" -f2 > sp.id
comm -23 <(sort sp.id) <(sort all_kunitz.id) > sp_negs.ids

2. Pull out from a fasta file only the sequences of the IDs list using a python script (`get_seq.py`)

In [ ]:
python3 get_seq.py sp_negs.ids uniprot_sprot.fasta sp_negs.fasta

FILE: `sp_negs.fasta` (this is the negative set)

__4.3 Generate random subsets of positive and negative sequences__

1. Randomize the IDs file (Both positive and negative) and divide both in half

In [ ]:
# Randomize
sort -R sp_negs.ids > random_sp_negs.ids
sort -R to_keep.ids > random_ok_kunitz.ids

# Cut in half
head -n 183 random_ok_kunitz.ids > pos_1.ids
tail -n 183 random_ok_kunitz.ids > pos_2.ids

head -n 286417 random_sp_negs.ids > neg_1.ids
tail -n 286417 random_sp_negs.ids > neg_2.ids

2. Pull out from all the UniProtKB Reviewed (Swiss-Prot) only the sequences of the IDs list using a python script (`get_seq.py`)

In [ ]:
python3 get_seq.py pos_1.ids uniprot_sprot.fasta  pos_1.fasta
python3 get_seq.py pos_2.ids uniprot_sprot.fasta  pos_2.fasta
python3 get_seq.py neg_1.ids uniprot_sprot.fasta  neg_1.fasta
python3 get_seq.py neg_2.ids uniprot_sprot.fasta  neg_2.fasta

### 5. Test the model
-------------------------------------------------------------------------
Now I test the model with `hmmsearch`, an HMMER program for search sequences that correspond to the HMM model
- `--max` = turns off all the heuristics for cutting off distantly related proteins
- `-tblout` = returns the output in tabular form
- `-Z` = for normalizing the e-value output



In [ ]:
hmmsearch -Z 1000 --max --tblout neg_2.out structural_model.hmm neg_2.fasta
hmmsearch -Z 1000 --max --tblout neg_1.out structural_model.hmm neg_1.fasta
hmmsearch -Z 1000 --max --tblout pos_2.out structural_model.hmm pos_2.fasta
hmmsearch -Z 1000 --max --tblout pos_1.out structural_model.hmm pos_1.fasta

The next step is to evaluate the performance of the model computing a confusion matrix but first I have to transforme the `.out` file obtained with `hmmsearch` in `.class` file compatible with the python script.

The following commands process the tabular output (`.out`) files generated by `hmmsearch`, extracting relevant information to evaluate the performance of the HMM model.

- `grep -v "^#"` removes comment lines from the output (those starting with `#`).
- `awk`:
  - Uses `split($1,a,"|")` to extract the sequence ID (second field in the pipe-separated name).
  - Assigns a **label**: `1` for positive sets, `0` for negative sets.
  - Extracts the **score** and **e-value** (fields `$5` and `$8`, respectively).
- The result is a tab-delimited `.class` file with four columns:
  - `Sequence ID`, `Label`, `Score`, `E-value`

In [ ]:
grep -v "^#" pos_1.out | awk '{split($1,a,"|"); print a[2]"\t1\t"$5"\t"$8}' > pos_1.class
grep -v "^#" pos_2.out | awk '{split($1,a,"|"); print a[2]"\t1\t"$5"\t"$8}' > pos_2.class
grep -v "^#" neg_1.out | awk '{split($1,a,"|"); print a[2]"\t0\t"$5"\t"$8}' > neg_1.class
grep -v "^#" neg_2.out | awk '{split($1,a,"|"); print a[2]"\t0\t"$5"\t"$8}' > neg_2.class

__Recovering false negatives excluded by HMMER's default threshold__

By default, HMMER ignores all hits with an E-value greater than 10, considering them non-significant.
However, for the purpose of building a complete evaluation set (including true negatives and potential false negatives), we manually added back sequences that were not reported by `hmmsearch` due to this threshold.

To do this, we created a set of protein sequences assumed to be **non-matching (negatives)** and manually assigned them an **E-value of 10.0**.
This ensures they are clearly **above the detection threshold** and can be included in our evaluation as sequences that should **not** match the model.

This allows us to:
- Simulate how the model behaves when encountering truly unrelated sequences.
- Include potential **false negatives** that were missed due to the E-value filter.
- Build a balanced `.class` file with both predicted positives and negatives.

These artificially added entries help improve the completeness and fairness of the evaluation using the confusion matrix.



In [ ]:
comm -23 <(sort neg_1.ids) <(cut -f1 neg_1.class | sort) | awk '{print $1"\t0\t10.0\t10.0"}' >> neg_1_hits.class
comm -23 <(sort neg_2.ids) <(cut -f1 neg_2.class | sort) | awk '{print $1"\t0\t10.0\t10.0"}' >> neg_2_hits.class

Merge the set of positive (`pos_1.class`, `pos_2.class`) with the two recovered negative sets (`neg_1_hits.class`, `neg_2_hits.class`).

In [ ]:
cat pos_1.class neg_1_hits.class > set_1.class
cat pos_2.class neg_2_hits.class > set_2.class

FILE:
- `set_1.class`
- `set_2.class`

### 6. 2-Fold Cross-Validation Strategy
------------------------

To evaluate the performance of the HMM-based classifier and select an optimal E-value threshold, I apply a 2-fold cross-validation.

1. **Training on Set 1, Testing on Set 2**
   - Apply a range of E-value thresholds on Set 1.
   - For each threshold, compute the confusion matrix (TP, FP, TN, FN) and calculate evaluation metrics such as:
     - Accuracy
     - Recall
     - Matthews Correlation Coefficient (MCC)
   - Select the E-value threshold that maximizes MCC on Set 1.
   - This threshold is then applied to classify Set 2.
   - The resulting performance on Set 2 is recorded.

3. **Training on Set 2, Testing on Set 1**
   - The same process is repeated in reverse:
     - Find the best threshold on Set 2
     - Apply it to classify Set 1
     - Evaluate performance on Set 1

4. **Global Assessment**
   - Metrics from both folds are combined to give a more robust and unbiased estimate of the model's performance.

This procedure ensures that the threshold selection is unbiased and not overfitted to a specific subset of the data.

In [ ]:
python3 script_python/performance_crossvalidation.py set_1.class set_2.class > hmm_result.txt

Detailed results on the file `hmm_result.txt`

### 7. E-value Threshold Selection for Error Analysis
----------

To evaluate false positives and false negatives, we applied a classification threshold on the E-value scores produced by HMMER.

Multiple E-value thresholds were tested during the model evaluation phase to identify the one that maximized classification performance. This was done using Matthews Correlation Coefficient (MCC) as the objective metric during 2-fold cross-validation.

- **Optimal threshold for Set 1**: 1e-3
- **Optimal threshold for Set 2**: 1e-2

Based on these results, we selected **1e-3** as a representative and conservative threshold for downstream analysis of classification errors (false positives and false negatives).

This threshold was then applied to `set_1.class and set_2.class` files to extract:
- **False negatives**: true Kunitz proteins (label = 1) with E-value ≥ 1e-3.
- **False positives**: non-Kunitz proteins (label = 0) with E-value < 1e-3.


In [ ]:
# False Negative
awk -v t=1e-5 '$2 == 1 && $4 >= t' set_1.class
awk -v t=1e-5 '$2 == 1 && $4 >= t' set_2.class

# False Positive
awk -v t=1e-5 '$2 == 0 && $4 <  t' set_1.class
awk -v t=1e-5 '$2 == 0 && $4 <  t' set_2.class

### Example of classification results (label = 1)

| Sequence ID | Label | Score     | E-value   |
|-------------|--------|-----------|------------|
| Q8WPG5      | 1      | 0.00025   | 0.002      |
| D3GGZ8      | 1      | 0.0096    | 0.0096     |

→ Proteins with E-value **greater than 1e-3** are classified as **negative** by the model.

**Interpretation**:
- Q8WPG5 and D3GGZ8 are misclassified as negatives (**false negatives**) despite being labeled as positive
